<a href="https://colab.research.google.com/github/mbayekebe/Coursera_Capstone/blob/master/ClusteringTorontoNeighborhoodG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Central Toronto</font></h1>

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [0]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.18.1               |             py_0          51 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          84 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.18.1-py_0 conda-forge


geopy-1.18.1         | 51 KB     | ##################################### | 100% 
geographiclib-1.49   | 32 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Solving environment: done

## Package Plan ##

  envi

<a id='item1'></a>

## 1. Download and Explore Dataset

#### Load and explore the data

In [0]:
#read csv file of geocoded Toronto neighborhood
cludf = pd.read_csv('GeocodedPostalCodesToronto.csv' , index_col=0) 

# selecting Central Toronto borough only for exploring and clustering 
torontodf = cludf.loc[cludf['Borough'] == 'Central Toronto' ]

In [0]:
# Let's get the geographical coordinates of Central Toronto 

address = 'Central Toronto, CA' 

geolocator = Nominatim()
 
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Central Toronto are {}, {}.'.format(latitude, longitude))


/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  """


The geograpical coordinate of Central Toronto are 43.653963, -79.387207.


#### Create a map of Toronto with neighborhoods superimposed on top.

In [0]:
# create map of Central Toronto using latitude and longitude values
map_centraltoronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(torontodf['Latitude'], torontodf['Longitude'], torontodf['Borough'], torontodf['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_centraltoronto)  
    
map_centraltoronto

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

#### Define Foursquare Credentials and Version

In [0]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

#### Let's explore the first neighborhood in our dataframe.

Get the neighborhood's name.

In [0]:
torontodf.loc[61, 'Neighborhood'] 

'Lawrence Park'

Get the neighborhood's latitude and longitude values.

In [0]:
neighborhood_latitude = torontodf.loc[61, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = torontodf.loc[61, 'Longitude'] # neighborhood longitude value

neighborhood_name = torontodf.loc[61, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Lawrence Park are 43.7280205, -79.3887901.


# Now, let's get the top 100 venues that are in Lawrence Park within a radius of 500 meters.

First, let's create the GET request URL. Name your URL **url**.

In [0]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id=&client_secret=&v=20180604&ll=43.7280205,-79.3887901&radius=500&limit=100'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
#url # display URL

Send the GET request and examine the resutls

In [0]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c2d120af594df03ec267655'},
 'response': {'headerLocation': 'Toronto',
  'headerFullLocation': 'Toronto',
  'headerLocationGranularity': 'city',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 43.7325205045, 'lng': -79.3825744605273},
   'sw': {'lat': 43.7235204955, 'lng': -79.3950057394727}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '50e6da19e4b0d8a78a0e9794',
       'name': 'Lawrence Park Ravine',
       'location': {'address': '3055 Yonge Street',
        'crossStreet': 'Lawrence Avenue East',
        'lat': 43.72696303913755,
        'lng': -79.39438246708775,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.72696303913755,
          'lng': -79.39438246708775}],
        'distance': 465,
        'c

From the Foursquare lab in the previous module, we know that all the information is in the *items* key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.

In [0]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a *pandas* dataframe.

In [0]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()


,name,categories,lat,lng
0,Lawrence Park Ravine,Park,43.726963,-79.394382
1,The Photo School – Toronto,Photography Studio,43.730429,-79.388767
2,Zodiac Swim School,Swim School,43.728532,-79.382860
3,TTC Bus #162 - Lawrence-Donway,Bus Line,43.728026,-79.382805


And how many venues were returned by Foursquare?

In [0]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


<a id='item2'></a>

## 2. Explore Neighborhoods in Central Toronto

#### Let's create a function to repeat the same process to all the neighborhoods in Central Toronto

In [0]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *manhattan_venues*.

In [0]:
# type your answer here

centraltoronto_venues = getNearbyVenues(names=torontodf['Neighborhood'],
                                   latitudes=torontodf['Latitude'],
                                   longitudes=torontodf['Longitude']
                                  )



Lawrence Park
Roselawn
Davisville North
Forest Hill North,Forest Hill West
North Toronto West
The Annex,North Midtown,Yorkville
Davisville
Moore Park,Summerhill East
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West


#### Let's check the size of the resulting dataframe

In [0]:
print(centraltoronto_venues.shape)
centraltoronto_venues.head()

(111, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Lawrence Park,43.728020,-79.388790,Lawrence Park Ravine,43.726963,-79.394382,Park
1,Lawrence Park,43.728020,-79.388790,The Photo School – Toronto,43.730429,-79.388767,Photography Studio
2,Lawrence Park,43.728020,-79.388790,Zodiac Swim School,43.728532,-79.382860,Swim School
3,Lawrence Park,43.728020,-79.388790,TTC Bus #162 - Lawrence-Donway,43.728026,-79.382805,Bus Line
4,Roselawn,43.711695,-79.416936,Dr.Paul Hodges MIP,43.710634,-79.415810,Health & Beauty Service


Let's check how many venues were returned for each neighborhood

In [0]:
centraltoronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Davisville,33,33,33,33,33,33
Davisville North,7,7,7,7,7,7
"Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West",16,16,16,16,16,16
"Forest Hill North,Forest Hill West",4,4,4,4,4,4
Lawrence Park,4,4,4,4,4,4
"Moore Park,Summerhill East",1,1,1,1,1,1
North Toronto West,18,18,18,18,18,18
Roselawn,2,2,2,2,2,2
"The Annex,North Midtown,Yorkville",26,26,26,26,26,26


#### Let's find out how many unique categories can be curated from all the returned venues

In [0]:
print('There are {} uniques categories.'.format(len(centraltoronto_venues['Venue Category'].unique())))

There are 60 uniques categories.


<a id='item3'></a>

## 3. Analyze Each Neighborhood

In [0]:
# one hot encoding
toronto_onehot = pd.get_dummies(centraltoronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = centraltoronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,American Restaurant,BBQ Joint,Bagel Shop,Breakfast Spot,Brewery,Burger Joint,Bus Line,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Convenience Store,Cosmetics Shop,Dessert Shop,Diner,Farmers Market,Fast Food Restaurant,Flower Shop,Food & Drink Shop,French Restaurant,Fried Chicken Joint,Garden,Gourmet Shop,Greek Restaurant,Gym,Gym / Fitness Center,Health & Beauty Service,History Museum,Hotel,Indian Restaurant,Indoor Play Area,Italian Restaurant,Jewelry Store,Jewish Restaurant,Light Rail Station,Liquor Store,Mexican Restaurant,Park,Pharmacy,Photography Studio,Pizza Place,Playground,Pub,Rental Car Location,Restaurant,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Spa,Sporting Goods Shop,Sports Bar,Supermarket,Sushi Restaurant,Swim School,Thai Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoga Studio
0,Lawrence Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Lawrence Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Lawrence Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,Lawrence Park,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Roselawn,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


And let's examine the new dataframe size.

In [0]:
toronto_onehot.shape

(111, 61)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [0]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,American Restaurant,BBQ Joint,Bagel Shop,Breakfast Spot,Brewery,Burger Joint,Bus Line,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Convenience Store,Cosmetics Shop,Dessert Shop,Diner,Farmers Market,Fast Food Restaurant,Flower Shop,Food & Drink Shop,French Restaurant,Fried Chicken Joint,Garden,Gourmet Shop,Greek Restaurant,Gym,Gym / Fitness Center,Health & Beauty Service,History Museum,Hotel,Indian Restaurant,Indoor Play Area,Italian Restaurant,Jewelry Store,Jewish Restaurant,Light Rail Station,Liquor Store,Mexican Restaurant,Park,Pharmacy,Photography Studio,Pizza Place,Playground,Pub,Rental Car Location,Restaurant,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Spa,Sporting Goods Shop,Sports Bar,Supermarket,Sushi Restaurant,Swim School,Thai Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoga Studio
0,Davisville,0.000000,0.000000,0.000000,0.000000,0.030303,0.030303,0.00,0.060606,0.000000,0.000000,0.060606,0.0000,0.000000,0.090909,0.030303,0.030303,0.000000,0.000000,0.000000,0.000000,0.030303,0.0,0.030303,0.030303,0.030303,0.000000,0.0,0.000000,0.000000,0.030303,0.030303,0.060606,0.00,0.000000,0.000,0.000000,0.000000,0.030303,0.030303,0.00,0.060606,0.0,0.000000,0.000000,0.030303,0.000000,0.090909,0.060606,0.000000,0.000000,0.0000,0.0000,0.060606,0.00,0.030303,0.030303,0.00,0.000000,0.0000,0.000000
1,Davisville North,0.000000,0.000000,0.000000,0.142857,0.000000,0.142857,0.00,0.000000,0.000000,0.142857,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.142857,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.142857,0.000000,0.000000,0.000000,0.00,0.000000,0.000,0.000000,0.000000,0.142857,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.142857,0.000000,0.000000,0.000000,0.0000,0.0000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.0000,0.000000
2,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",0.062500,0.000000,0.062500,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.125000,0.0625,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.062500,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.125,0.000000,0.000000,0.000000,0.000000,0.00,0.062500,0.0,0.125000,0.000000,0.000000,0.000000,0.000000,0.000000,0.062500,0.000000,0.0625,0.0625,0.062500,0.00,0.000000,0.000000,0.00,0.000000,0.0625,0.000000
3,"Forest Hill North,Forest Hill West",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.25,0.000000,0.000,0.000000,0.000000,0.250000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0000,0.250000,0.00,0.000000,0.000000,0.25,0.000000,0.0000,0.000000
4,Lawrence Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000,0.000000,0.000000,0.250000,0.000000,0.25,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0000,0.000000,0.25,0.000000,0.000000,0.00,0.000000,0.0000,0.000000
5,"Moore Park,Summerhill East",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.0000

#### Let's confirm the new size

In [0]:
toronto_grouped.shape

(9, 61)

#### Let's print each neighborhood along with the top 5 most common venues

In [0]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Davisville----
                venue  freq
0      Sandwich Place  0.09
1        Dessert Shop  0.09
2         Pizza Place  0.06
3  Italian Restaurant  0.06
4                Café  0.06


----Davisville North----
               venue  freq
0     Clothing Store  0.14
1       Burger Joint  0.14
2  Food & Drink Shop  0.14
3               Park  0.14
4              Hotel  0.14


----Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West----
                 venue  freq
0          Coffee Shop  0.12
1                  Pub  0.12
2   Light Rail Station  0.12
3  Fried Chicken Joint  0.06
4          Pizza Place  0.06


----Forest Hill North,Forest Hill West----
                 venue  freq
0        Jewelry Store  0.25
1                Trail  0.25
2     Sushi Restaurant  0.25
3                 Park  0.25
4  American Restaurant  0.00


----Lawrence Park----
                 venue  freq
0             Bus Line  0.25
1                 Park  0.25
2          Swim School  0.25
3   Photography Stu

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [0]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [0]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Davisville,Sandwich Place,Dessert Shop,Café,Pizza Place,Italian Restaurant,Seafood Restaurant,Sushi Restaurant,Coffee Shop,Indian Restaurant,Pharmacy
1,Davisville North,Clothing Store,Food & Drink Shop,Hotel,Breakfast Spot,Burger Joint,Sandwich Place,Park,Yoga Studio,French Restaurant,Fried Chicken Joint
2,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",Pub,Light Rail Station,Coffee Shop,Sports Bar,Vietnamese Restaurant,Fried Chicken Joint,Pizza Place,Convenience Store,Spa,American Restaurant
3,"Forest Hill North,Forest Hill West",Trail,Jewelry Store,Sushi Restaurant,Park,Yoga Studio,Fried Chicken Joint,Fast Food Restaurant,Flower Shop,Food & Drink Shop,French Restaurant
4,Lawrence Park,Swim School,Bus Line,Park,Photography Studio,Yoga Studio,Farmers Market,Gym / Fitness Center,Gym,Greek Restaurant,Gourmet Shop
5,"Moore Park,Summerhill East",Playground,Yoga Studio,Hotel,Health & Beauty Service,Gym / Fitness Center,Gym,Greek Restaurant,Gourmet Shop,Garden,Fried Chicken Joint
6,North Toronto West,Coffee Shop,Sporting Goods Shop,Yoga Studio,Clothing Store,Fast Food Restaurant,Mexican Restaurant,Park,Diner,Dessert Shop,Gym / Fitness Center
7,Roselawn,Health & Beauty Service,Garden,Yoga Studio,Vietnamese Restaurant,History Museum,Gym / Fitness Center,Gym,Greek Restaurant,Gourmet Shop,Fried Chicken Joint
8,"The Annex,North Midtown,Yorkville",Coffee Shop,Café,Sandwich Place,Pizza Place,Indian Restaurant,Pub,BBQ Joint,Burger Joint,Cosmetics Shop,Flower Shop


<a id='item4'></a>

## 4. Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [0]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 4, 3, 2, 0, 1, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [0]:
toronto_merged = torontodf

# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
61,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Swim School,Bus Line,Park,Photography Studio,Yoga Studio,Farmers Market,Gym / Fitness Center,Gym,Greek Restaurant,Gourmet Shop
62,M5N,Central Toronto,Roselawn,43.711695,-79.416936,0,Health & Beauty Service,Garden,Yoga Studio,Vietnamese Restaurant,History Museum,Gym / Fitness Center,Gym,Greek Restaurant,Gourmet Shop,Fried Chicken Joint
67,M4P,Central Toronto,Davisville North,43.712751,-79.390197,0,Clothing Store,Food & Drink Shop,Hotel,Breakfast Spot,Burger Joint,Sandwich Place,Park,Yoga Studio,French Restaurant,Fried Chicken Joint
68,M5P,Central Toronto,"Forest Hill North,Forest Hill West",43.696948,-79.411307,4,Trail,Jewelry Store,Sushi Restaurant,Park,Yoga Studio,Fried Chicken Joint,Fast Food Restaurant,Flower Shop,Food & Drink Shop,French Restaurant
73,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,3,Coffee Shop,Sporting Goods Shop,Yoga Studio,Clothing Store,Fast Food Restaurant,Mexican Restaurant,Park,Diner,Dessert Shop,Gym / Fitness Center


Finally, let's visualize the resulting clusters

In [0]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='item5'></a>

## 5. Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

#### Cluster 1

In [0]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
61,Central Toronto,0,Swim School,Bus Line,Park,Photography Studio,Yoga Studio,Farmers Market,Gym / Fitness Center,Gym,Greek Restaurant,Gourmet Shop
62,Central Toronto,0,Health & Beauty Service,Garden,Yoga Studio,Vietnamese Restaurant,History Museum,Gym / Fitness Center,Gym,Greek Restaurant,Gourmet Shop,Fried Chicken Joint
67,Central Toronto,0,Clothing Store,Food & Drink Shop,Hotel,Breakfast Spot,Burger Joint,Sandwich Place,Park,Yoga Studio,French Restaurant,Fried Chicken Joint
79,Central Toronto,0,Sandwich Place,Dessert Shop,Café,Pizza Place,Italian Restaurant,Seafood Restaurant,Sushi Restaurant,Coffee Shop,Indian Restaurant,Pharmacy
86,Central Toronto,0,Pub,Light Rail Station,Coffee Shop,Sports Bar,Vietnamese Restaurant,Fried Chicken Joint,Pizza Place,Convenience Store,Spa,American Restaurant


#### Cluster 2

In [0]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
83,Central Toronto,1,Playground,Yoga Studio,Hotel,Health & Beauty Service,Gym / Fitness Center,Gym,Greek Restaurant,Gourmet Shop,Garden,Fried Chicken Joint


#### Cluster 3

In [0]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
74,Central Toronto,2,Coffee Shop,Café,Sandwich Place,Pizza Place,Indian Restaurant,Pub,BBQ Joint,Burger Joint,Cosmetics Shop,Flower Shop


#### Cluster 4

In [0]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
73,Central Toronto,3,Coffee Shop,Sporting Goods Shop,Yoga Studio,Clothing Store,Fast Food Restaurant,Mexican Restaurant,Park,Diner,Dessert Shop,Gym / Fitness Center


#### Cluster 5

In [0]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
68,Central Toronto,4,Trail,Jewelry Store,Sushi Restaurant,Park,Yoga Studio,Fried Chicken Joint,Fast Food Restaurant,Flower Shop,Food & Drink Shop,French Restaurant
